# 🌊🌊 **SWELLS** — *Supervised Finetuner*
[![Paper](http://img.shields.io/badge/Arxiv:2503.09454-B31B1B.svg)](https://arxiv.org/abs/2503.09454)
[![GitHub](https://img.shields.io/badge/GitHub-Repo-181717?logo=github)](https://github.com/mmarmonier/SWELLS)

This notebook performs **Supervised Fine-Tuning (SFT)** on an LLM using a **ciphered conlang translation training set** with a mix of CoT demonstrations and spontaneous translation examples.  

It fine-tunes a model via the OpenAI fine-tuning API, aiming to enhance an LLM's ability to **explicitly learn grammar rules** using metalinguistic explanations found in grammar book excerpts.  

🚨 **WARNING:** Running this notebook requires an OpenAI API key and can be costly for large training sets. Cost estimations are provided but current rates should be carefully checked: https://platform.openai.com/docs/pricing.

# API key

In [ ]:
# Replace the XXXX string with an OpenAI API key.
openAI_API_key = 'XXXX'

# Data (tiny sample for demonstration purposes)

In [ ]:
!wget https://github.com/mmarmonier/SWELLS/releases/download/v1.0/SWELLS_dataset.7z

In [ ]:
!7z x /content/SWELLS_dataset.7z -p"SWELLS"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 299744346 bytes (286 MiB)

Extracting archive: /content/drive/MyDrive/SWELLS_dataset.7z
--
Path = /content/drive/MyDrive/SWELLS_dataset.7z
Type = 7z
Physical Size = 299744346
Headers Size = 1754
Method = LZMA2:24 7zAES
Solid = +
Blocks = 12

  0%      0% 14 - SWELLS_Dataset/Dev/dev_French.jsonl                                               0% 15 - SWELLS_Dataset/Dev/dev_Latin.jsonl                                              1% 15 - SWELL

In [ ]:
!head -n 10 /content/SWELLS_Dataset/Train/train_example.jsonl > train.jsonl

In [ ]:
!head -n 10 /content/SWELLS_Dataset/Dev/dev_French.jsonl > dev.jsonl

# Checking and uploading the datasets

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00


## Dev set

In [ ]:
dev_data_path = "/content/dev.jsonl"

import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = dev_data_path

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 65,536 token limit, they will be truncated during fine-tuning")
print("VALIDATION TOKENS ARE FREE OF CHARGE ON THE OPENAI FINETUNING API (for the time being, december 2024)")

Num examples: 10
First example:
{'role': 'system', 'content': 'You are an expert linguist and translator.'}
{'role': 'user', 'content': 'Ltesèaij is a recently devised conlang. You are to translate the following English segment into Ltesèaij with the help of a few dictionary entries and excerpts from a grammar book.\n\nHere is the text segment you must translate:\nthe gusts that I buried\n\nHere are a few dictionary entries that may be of use to you; note that each entry follows the format: English lemma : Ltesèaij equivalent (grammatical gender and/or part of speech).\nbury (v.) : iwñséjsèa (2nd group v.) \ngust : avàzjzi (fem. n.)\n\nAnd here are relevant excerpts from a grammar book: \n\nBeginning of Ltesèaij Grammar Book Excerpts\n\n\t\t\t\t\t\t\t\t\t\t\nWORD ORDER\n\t\t\t\t\t\t\t\t\t\t\n\nObject-verb-subject (OVS) is the primary word order in Ltesèaij, distinguishing it from many other languages of the world.\n\nFor example: añvéwnéñfe zvà aiségàz’b (\'I love music\').\n\n§§§§§§§§

In [ ]:
# Dev
from openai import OpenAI

client = OpenAI(api_key=openAI_API_key)

dev_file = client.files.create(
  file=open(dev_data_path, "rb"),
  purpose="fine-tune"
)

dev_file_id = dev_file.id

In [ ]:
print(dev_file_id)

file-95k9h6y14TrdJu8RDftnru


## Training set

In [ ]:
train_data_path = f"train.jsonl"

In [ ]:
!head train.jsonl

{"messages": [{"role": "system", "content": "You are an expert linguist and translator."}, {"role": "user", "content": "Àjçvèlfag is a recently devised conlang. You are to translate the following English segment into Àjçvèlfag with the help of a few dictionary entries and excerpts from a grammar book.\n\nHere is the text segment you must translate:\nthe feasts\n\nHere are a few dictionary entries that may be of use to you; note that each entry follows the format: English lemma : Àjçvèlfag equivalent (grammatical gender and/or part of speech).\nfeast : acàzexanf (masc. n.)\n\nAnd here are relevant excerpts from a grammar book: \n\nBeginning of Àjçvèlfag Grammar Book Excerpts\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\nARTICLES AND DETERMINERS\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\nUnlike in English, articles and determiners are almost always used with common nouns in Àjçvèlfag. They reflect the gender (masculine or feminine) and number (singular or plural) of the 

In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = train_data_path

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 65,536 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 65536

TARGET_EPOCHS = 1
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
#print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
cost_per_million_tokens = 3.000  # Update with the cost per million training tokens
total_cost = (n_epochs * n_billing_tokens_in_dataset / 1_000_000) * cost_per_million_tokens
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens, costing approximately ${total_cost:.2f}")


Num examples: 10
First example:
{'role': 'system', 'content': 'You are an expert linguist and translator.'}
{'role': 'user', 'content': 'Àjçvèlfag is a recently devised conlang. You are to translate the following English segment into Àjçvèlfag with the help of a few dictionary entries and excerpts from a grammar book.\n\nHere is the text segment you must translate:\nthe feasts\n\nHere are a few dictionary entries that may be of use to you; note that each entry follows the format: English lemma : Àjçvèlfag equivalent (grammatical gender and/or part of speech).\nfeast : acàzexanf (masc. n.)\n\nAnd here are relevant excerpts from a grammar book: \n\nBeginning of Àjçvèlfag Grammar Book Excerpts\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\nARTICLES AND DETERMINERS\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\nUnlike in English, articles and determiners are almost always used with common nouns in Àjçvèlfag. They reflect the gender (masculine or feminine) and number (singular o

In [ ]:
train_data_path

'train.jsonl'

In [ ]:
# Train
from openai import OpenAI

client = OpenAI(api_key=openAI_API_key)

train_file = client.files.create(
  file=open(train_data_path, "rb"),
  purpose="fine-tune"
)

train_file_id = train_file.id

In [ ]:
print(train_file_id)

file-Qswbxuoo3UWL45nAfDkCv1


# Launching FT job

In [ ]:
# Finetuned base
base_model_identifier = "gpt-4o-mini-2024-07-18"

In [ ]:
# Defining finetuned model naming suffix
ft_partition_name = f"_finetuned_on_very_few_instances"
assert len(ft_partition_name) < 64, "Suffix should have fewer than 65 characters"

In [ ]:
from openai import OpenAI


client.fine_tuning.jobs.create(
  training_file=train_file_id,
  validation_file=dev_file_id,
  model=base_model_identifier,
  suffix=ft_partition_name,
  hyperparameters={
    "n_epochs":1,
  }
)

FineTuningJob(id='ftjob-cHwjq0RteGzRSneJbiTgGZQQ', created_at=1741962850, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-xFBR7bDUgyCgOidVL3xbTyqO', result_files=[], seed=1610989548, status='validating_files', trained_tokens=None, training_file='file-Qswbxuoo3UWL45nAfDkCv1', validation_file='file-95k9h6y14TrdJu8RDftnru', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='_finetuned_on_very_few_instances', metadata=None)

In [ ]:
from openai import OpenAI

client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-cHwjq0RteGzRSneJbiTgGZQQ', created_at=1741962850, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-xFBR7bDUgyCgOidVL3xbTyqO', result_files=[], seed=1610989548, status='validating_files', trained_tokens=None, training_file='file-Qswbxuoo3UWL45nAfDkCv1', validation_file='file-95k9h6y14TrdJu8RDftnru', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='_finetuned_on_very_few_instances', metadata=None)], object='list', has_more=True)